# S3DXRD processing first step: dataset creation, segmentation and peak labelling

Written by James Ball, Haixing Fang and Jon Wright

Last updated: 21/02/2025

Outside ESRF: download [install_ImageD11_from_git.py](https://github.com/FABLE-3DXRD/ImageD11/tree/master/ImageD11/nbGui/install_ImageD11_from_git.py), and update the path in the next cell:

In [ ]:
exec(open('/data/id11/nanoscope/install_ImageD11_from_git.py').read())

In [ ]:
# this cell is tagged with 'parameters'
# to view the tag, select the cell, then find the settings gear icon (right or left sidebar) and look for Cell Tags

PYTHONPATH = setup_ImageD11_from_git( ) # ( os.path.join( os.environ['HOME'],'Code'), 'ImageD11_git' )

# Experts : update these files for your detector if you need to
maskfile = "/data/id11/nanoscope/Eiger/eiger_mask_E-08-0144_20240205.edf"
e2dxfile = "/data/id11/nanoscope/Eiger/e2dx_E-08-0144_20240205.edf"
e2dyfile = "/data/id11/nanoscope/Eiger/e2dy_E-08-0144_20240205.edf"
detector = 'eiger'
omegamotor = 'rot_center'
dtymotor = 'dty'

# Default segmentation options
options = { 'cut' : 1, 'pixels_in_spot' : 3, 'howmany' : 100000 }
normalise_intensities_to_monitor = True
monitor_name = 'fpico6'

# EXPERTS: These can be provided as papermill parameters. Users, leave these as None for now...
dataroot = None
analysisroot = None
sample = None
dataset = None

In [ ]:
# Import needed packages
import pprint
import numpy as np
from matplotlib import pyplot as plt

import ImageD11.sinograms.dataset
import ImageD11.sinograms.lima_segmenter
import ImageD11.sinograms.assemble_label
import ImageD11.sinograms.properties
import ImageD11.nbGui.nb_utils as utils
from ImageD11.nbGui import segmenter_gui

%matplotlib ipympl

In [ ]:
# Set up the file paths. Edit this if you are not at ESRF or not using the latest data policy.
if dataroot is None:
    dataroot, analysisroot = segmenter_gui.guess_ESRF_paths() 

if len(dataroot)==0:
    print("Please fix in the dataroot and analysisroot folder names above!!")
    
print('dataroot =',repr(dataroot))
print('analysisroot =',repr(analysisroot))

In [ ]:
# List the samples available:
segmenter_gui.printsamples(dataroot)

In [ ]:
# USER: Decide which sample
if sample is None:
    sample = 'FeAu_0p5_tR_nscope'

In [ ]:
# List the datasets for that sample:
segmenter_gui.printdatasets( dataroot, sample )

In [ ]:
# USER: Decide which dataset
if dataset is None:
    dataset = "top_100um"

## Example demo to see how a frame is segmented for one dataset:

In [ ]:
# create ImageD11 dataset object
ds = ImageD11.sinograms.dataset.DataSet(dataroot=dataroot,
                                        analysisroot=analysisroot,
                                        sample=sample,
                                        dset=dataset,
                                        detector=detector,
                                        omegamotor=omegamotor,
                                        dtymotor=dtymotor
                                       )
ds.import_all()  # Can use scans = [f'{scan}.1' for scan in range(1,102)] )
ds.maskfile = maskfile
ds.e2dxfile = e2dxfile
ds.e2dyfile = e2dyfile
ds.save()

In [ ]:
ui = segmenter_gui.SegmenterGui(ds, **options )

In [ ]:
options = ui.getopts()

# Sparse file creation

In [ ]:
# create batch file to send to SLURM cluster
sbat = ImageD11.sinograms.lima_segmenter.setup(ds.dsfile, **ui.getopts(), pythonpath=PYTHONPATH)
if sbat is None:
    raise ValueError("This scan has already been segmented!")
print(sbat)

In [ ]:
utils.slurm_submit_and_wait(sbat, 60)

# Peak labeling

In [ ]:
# label sparse peaks

ImageD11.sinograms.assemble_label.main(ds.dsfile)

In [ ]:
# generate peaks table

ImageD11.sinograms.properties.main(ds.dsfile, options={'algorithm': 'lmlabel', 'wtmax': 70000, 'save_overlaps': False})

# Intensity normalisation
You can optionally normalise your observed intensities to a monitor column, such as a pico, if you had one in the beam path.

In [ ]:
fig, ax = plt.subplots(layout='constrained')
im, om_edges, dty_edges = ds.sinohist(np.log(ds.pk2d['sum_intensity']), ds.pk2d['omega'], ds.pk2d['dty'], return_edges=True)
pcm = ax.pcolormesh(om_edges, dty_edges, im.T)
ax.set(xlabel=r'$\omega~(\degree)$', ylabel='dty', title='Un-normalised sinogram of all 2D peaks')
cax = fig.colorbar(pcm, ax=ax, label='log(intensity)')
plt.show()

Let's look at an image of the monitor signal across (dty, omega)

In [ ]:
if normalise_intensities_to_monitor:
    monitor_per_frame = ds.get_monitor(monitor_name)

    fig, ax = plt.subplots(layout='constrained')
    pcm = ax.pcolormesh(om_edges, dty_edges, monitor_per_frame)
    ax.set(xlabel=r'$\omega~(\degree)$', ylabel='dty', title='Monitor column')
    cax = fig.colorbar(pcm, ax=ax, label='Monitor')
    plt.show()

To normalise to a monitor signal, we need to choose a "reference" monitor value that we scale to.
A good choice may be `np.mean(monitor_per_frame)`

We then compute `scale_factor_per_frame = np.mean(monitor_per_frame) / monitor_per_frame`  
We then multiply the observed intensities by `scale_factor_per_frame`

You can choose which function to use to generate the "reference" monitor value, for example:

`ds.set_monitor(monitor_name='fpico6', ref_value_func=np.mean)`

In [ ]:
if normalise_intensities_to_monitor:
    # ensure no monitor currently set
    ds.monitor = None
    ds.monitor_ref = None
    ds.reset_peaks_cache()

    fig, ax = plt.subplots(layout='constrained')
    im, om_edges, dty_edges = ds.sinohist(ds.pk2d['sum_intensity'], ds.pk2d['omega'], ds.pk2d['dty'], return_edges=True)
    ax.plot(dty_edges, im.sum(axis=0), label='before scaling')
    
    # here we set the monitor
    ds.set_monitor(monitor_name, ref_value_func=np.mean)

    im, om_edges, dty_edges = ds.sinohist(ds.pk2d['sum_intensity'], ds.pk2d['omega'], ds.pk2d['dty'], return_edges=True)
    ax.plot(dty_edges, im.sum(axis=0), label='after scaling')
    ax.legend()
    ax.set(xlabel='dty', ylabel='intensity', title='Before vs after setting monitor')
    plt.show()
    
    # save the choice of monitor to disk
    ds.save()

# Finished segmenting!

You can now choose between two different indexing routes: tomographic (tomo) and point-by-point (pbp).  
Tomo gives you better grain shapes, but can't handle highly deformed samples.  
Point-by-point can only give you convex grain shapes (less accurate) but can handle high levels of deformation.  
Both techniques will join back together during the strain refinement stage (notebook 3).  
Therefore notebooks 4 and onwards should work from either the tomo or pbp route.